# ARM Acknowledgment 


**Autoloader set up**  
This Notebook sets up an Autoloader job that runs on a manual trigger to collect ack messages from the ack eventhubs


In [0]:
import json

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, LongType
from pyspark.sql.functions import col,from_json
import json

ack_schema = StructType([
    StructField("filename", StringType(), True),
    StructField("http_response", IntegerType(),True),
    StructField("timestamp", TimestampType(), True),
    StructField("http_message", StringType(), True)
])

In [0]:
## set up the configuration to allow the autoloader to connect to the source system

## Eventhub details
connection_string = dbutils.secrets.get("ingest00-meta002-sbox", "evh-apl-ack-uks-dlrm-01")
consumer_group = "$Default"
# Encrypt the connection string using the EventHubsUtils.encrypt method
encrypted_conn_str = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)
# Create a JSON string for starting positions for 10 partitions (0 through 9)
starting_positions = {str(i): "-1" for i in range(10)}
starting_positions_json = json.dumps(starting_positions)

ehConf = {
    "eventhubs.connectionString": encrypted_conn_str,
    "eventhubs.consumerGroup": consumer_group,
    # "eventhubs.startingPositions": starting_positions_json
}



eventhubdf = spark.readStream.format("eventhubs")\
    .options(**ehConf)\
        .load()

In [0]:
## set up the configuration to allow the autoloader to connect to the source system

## Eventhub details
connection_string = dbutils.secrets.get("ingest00-meta002-sbox", "evh-apl-ack-uks-dlrm-01")
consumer_group = "$Default"
# Encrypt the connection string using the EventHubsUtils.encrypt method
encrypted_conn_str = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connection_string)
# Create a JSON string for starting positions for 10 partitions (0 through 9)
starting_positions = {str(i): "-1" for i in range(10)}
starting_positions_json = json.dumps(starting_positions)

ehConf = {
    "eventhubs.connectionString": encrypted_conn_str,
    "eventhubs.consumerGroup": consumer_group,
    # "eventhubs.startingPositions": starting_positions_json
}


# display(spark.readStream.format("eventhubs").options(**ehConf).load())

In [0]:
parsed_df = (
    eventhubdf
    # 'body' is binary, so we cast to string (assuming UTF-8)
    .select(col("body").cast("string").alias("json_str"))
    .select(from_json(col("json_str"), ack_schema).alias("json_obj"))
    .select("json_obj.*")
)


# parsed_df.display()

In [0]:
ack_path = "/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/APPEALS/"

ack_file_name = f"{ack_path}FTA_ack_audit"

parsed_df.writeStream \
    .format("delta")\
        .option("checkpointLocation", "/mnt/autoLoaderSchema/ARMTD/ACK/ack")\
            .outputMode("append")\
                .trigger(availableNow=True)\
                .start(ack_file_name)

In [0]:
dbutils.fs.ls("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/")

In [0]:
spark.read.format("delta").load("/mnt/ingest00curatedsboxsilver/ARIADM/ARM/AUDIT/ARIATD/td_ack_audit").display()

In [0]:
# successful_files = result_df.filter(col("status") == "success").count()
# failed_files = result_df.filter(col("status") == "failure").count()

dbutils.notebook.exit("success")

## Appendix